In [ ]:
%tensorflow_version 2.x # Load tensorflow 2.x into notebook

In [ ]:
!pip install -q sklearn # install scikit-learn

In [ ]:
# from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [ ]:
list(df.groupby('Title'))

In [ ]:
dftrain = pd.read_csv('https://raw.githubusercontent.com/jcrecio/ml_titanic_survival/main/train.csv') # train data frame
dfeval = pd.read_csv('https://raw.githubusercontent.com/jcrecio/ml_titanic_survival/main/test.csv') # eval/test data frame
df = pd.concat([dftrain, dfeval], axis=0, sort=True)

df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.', expand=True)

# replacing some titles with more common ones
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr',
           'Don': 'Mr', 'Mme': 'Mrs', 'Jonkheer': 'Mr', 'Lady': 'Mrs',
           'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}
df.replace({'Title': mapping}, inplace=True)

df['Title'].value_counts()

# impute missing Age values using median of Title groups
title_ages = dict(df.groupby('Title')['Age'].median())

# create a column of the average ages
df['age_med'] = df['Title'].apply(lambda x: title_ages[x])

# replace all missing ages with the value in this column
df['Age'].fillna(df['age_med'], inplace=True, )
del df['age_med']

df['Embarked'].fillna(method='backfill', inplace=True) # Get rid of null values


class_fares = dict(df.groupby('Pclass')['Fare'].median()) # impute missing Fare values using median of Pclass groups
df['fare_med'] = df['Pclass'].apply(lambda x: class_fares[x]) # create a column of the average fares

# replace all missing fares with the value in this column
df['Fare'].fillna(df['fare_med'], inplace=True, )
del df['fare_med']

In [ ]:
df.drop(['Cabin', 'Name', 'Ticket', 'PassengerId'], axis=1, inplace=True) # Remove variables that will not be used

# Split df again in training and test datasets
X_train = df[pd.notnull(df['Survived'])].drop(['Survived'], axis=1) # training data has survived not null and remove the label
y_train = df[pd.notnull(df['Survived'])]['Survived'] # training data has survived not null and we only stay with the label
X_test = df[pd.isnull(df['Survived'])].drop(['Survived'], axis=1) # training data has survived null

In [ ]:
feature_columns = []

for feature_name in ["Pclass", "Sex", "SibSp", "Parch", "Embarked", "Title"]:
  vocabulary = df[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in ["Age", "Fare"]:
	feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
 
print(feature_columns)

In [ ]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

train_input_fn = make_input_fn(X_train, y_train, num_epochs=1, shuffle=True, batch_size=1)
pred_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_val,batch_size=len(X_val),shuffle=False)

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

In [ ]:
linear_est.train(train_input_fn)

In [ ]:
result = list(linear_est.predict(pred_input_fn))

In [ ]:
result